haarcascade face detection


*   서양인에 비해 동양인의 face detection 을 잘 못한다.
*   얼굴을 정면이 아닌 돌려져 있으면 face detection 을 잘 못한다.

In [1]:
# 내 구글 드라이브에 연동
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# 필요한 패키지와 모듈을 불러옴
import cv2
import numpy as np
import time
import io
import base64
from IPython.display import HTML

In [3]:
# OpenCV 버전을 확인
print("OpenCV version:")
print(cv2.__version__)

OpenCV version:
4.1.2


In [4]:
# Detection 하기 전에 원본 동영상을 Display
video = io.open('/content/gdrive/MyDrive/Haarcascade_Face_Detection/video/obama.mp4', 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<video width="50%" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4"/>
             </video>'''.format(encoded.decode('ascii')))

Output hidden; open in https://colab.research.google.com to view.

In [5]:
# 특성값이 미리 정의되어 있는 haarcascade xml 파일 위치 정의
face_cascade_name = '/content/gdrive/MyDrive/Haarcascade_Face_Detection/haarcascade_frontalface_alt.xml'     # 얼굴을 인식하는 feature 파일
eyes_cascade_name = '/content/gdrive/MyDrive/Haarcascade_Face_Detection/haarcascade_eye_tree_eyeglasses.xml' # 눈을 인식하는 feature 파일
file_name = '/content/gdrive/MyDrive/Haarcascade_Face_Detection/video/obama.mp4' # 원본 동영상          
output_name = 'output_video.mp4'  # detection 된 output 동영상

In [6]:
def detectAndDisplay(frame):
    # GRAY 이미지로 변환
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame_gray = cv2.equalizeHist(frame_gray)
    #-- Detect faces
    faces = face_cascade.detectMultiScale(frame_gray)
    for (x,y,w,h) in faces:
        center = (x + w//2, y + h//2)
        frame = cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 4)  # detection 된 얼굴에 사각형을 그린다.
        faceROI = frame_gray[y:y+h,x:x+w]   # frame_gray에서 방금 그린 얼굴 사각형 영역만 가져온다.(faceROI)
        # faceROI 에서 eyes 을 detection 한다.
        eyes = eyes_cascade.detectMultiScale(faceROI)
        for (x2,y2,w2,h2) in eyes:
            eye_center = (x + x2 + w2//2, y + y2 + h2//2)
            radius = int(round((w2 + h2)*0.25))  # 반지름 계산
            frame = cv2.circle(frame, eye_center, radius, (255, 0, 0 ), 4)  # detection 된 눈에 원을 그린다.
    
    # video 를 disk 에 output 하기 위해 writer 를 초기화한다.
    global writer
    if writer is None and output_name is not None:
        fourcc = cv2.VideoWriter_fourcc(*"DIVX")    # 코덱 정의
        writer = cv2.VideoWriter(output_name, fourcc, 20,
                (frame.shape[1], frame.shape[0]), True)
        
    # disk 에 frame 을 write 합니다.
    if writer is not None:
        writer.write(frame)

In [7]:
# face_cascade 인스턴스를 생성한다.
face_cascade = cv2.CascadeClassifier()
# eyes_cascade 인스턴스를 생성한다.
eyes_cascade = cv2.CascadeClassifier()

In [8]:
# cascades 파일을 불러온다.
if not face_cascade.load(cv2.samples.findFile(face_cascade_name)):
    print('--(!)Error loading face cascade')
    exit(0)
if not eyes_cascade.load(cv2.samples.findFile(eyes_cascade_name)):
         
    print('--(!)Error loading eyes cascade')
    exit(0)

In [9]:
# 원본 동영상에서 video stream을 읽어온다.
cap = cv2.VideoCapture(file_name)
writer = None
if not cap.isOpened:
    print('--(!)Error opening video capture')
    exit(0)
while True:
    ret, frame = cap.read()
    if frame is None:
        # close the video file pointers
        cap.release()
        # close the writer point
        writer.release()
        print('--(!) No captured frame -- Break!')
        break
    detectAndDisplay(frame)  # frame 을 보낸다.

--(!) No captured frame -- Break!
